# Library

In [14]:
import os
import glob
import random
import fnmatch

from shutil import copy

# Function

In [36]:
# Split data
def split_data(SOURCE_DIR, TRAINING_DIR, TEST_DIR, SPLIT_SIZE):
    contents = os.listdir(SOURCE_DIR)
    content_xml = [content for content in contents if content.endswith('.xml')]
    content_jpg = [content for content in contents if content.endswith('.jpg')]
    total_pic_num = len(content_jpg)
    
    training_jpg = random.sample(content_jpg, int(SPLIT_SIZE*total_pic_num))
    training_jpg_name = [file.split('.')[0] for file in training_jpg]
    training_xml = [xml for xml in content_xml if xml.split('.')[0] in training_jpg_name]
    test_jpg = [img for img in content_jpg if img.split('.')[0] not in training_jpg_name]
    test_xml = [xml for xml in content_xml if xml.split('.')[0] not in training_jpg_name]

    for img, xml in zip(training_jpg, training_xml):
        pic_dir = os.path.join(SOURCE_DIR, img)
        xml_dir = os.path.join(SOURCE_DIR, xml)
        copy(pic_dir, TRAINING_DIR)
        copy(xml_dir, TRAINING_DIR)
    
    for img, xml in zip(test_jpg, test_xml):
        pic_dir = os.path.join(SOURCE_DIR, img)
        xml_dir = os.path.join(SOURCE_DIR, xml)
        copy(pic_dir, TEST_DIR)
        copy(xml_dir, TEST_DIR)


# Path

In [37]:
images_path = os.path.join('Data', 'images')

mask_path = os.path.join(images_path, 'Mask')
no_mask_path = os.path.join(images_path, 'No_Mask')
not_wear_prop_path =  os.path.join(images_path, 'Not_Wear_Properly')

training_demo_images = os.path.join('workspace', 'training_demo', 'images')

train_path = os.path.join(training_demo_images, 'train')
test_path = os.path.join(training_demo_images, 'test')


# Split Data

In [38]:
split_data(mask_path, train_path, test_path, SPLIT_SIZE=0.8)
split_data(no_mask_path, train_path, test_path, SPLIT_SIZE=0.8)
split_data(not_wear_prop_path, train_path, test_path, SPLIT_SIZE=0.8)

# Convert xml to record 

In [51]:
# run below command in terminal
# command_train = python generate_tfrecord.py -x /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/images/train -l /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/annotations/label_map.pbtxt -o /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/annotations/train.record
# command_test = python generate_tfrecord.py -x /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/images/test -l /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/annotations/label_map.pbtxt -o /Users/michaelwork/Desktop/Project/Face_Mask_Detection/face_mask_detection/workspace/training_demo/annotations/test.record

# -------------------
# command_test = 'python generate_tfrecord.py -x {}/test -l {}/label_map.pbtxt -o {}/test.record'.format(train_path, anno_path, anno_path)

# !{command_train}
# !{command_test}

Traceback (most recent call last):
  File "generate_tfrecord.py", line 61, in <module>
    label_map = label_map_util.load_labelmap(args.labels_path)
  File "/Users/michaelwork/opt/anaconda3/envs/object-detection/lib/python3.8/site-packages/object_detection/utils/label_map_util.py", line 133, in load_labelmap
    label_map_string = fid.read()
  File "/Users/michaelwork/opt/anaconda3/envs/object-detection/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py", line 116, in read
    self._preread_check()
  File "/Users/michaelwork/opt/anaconda3/envs/object-detection/lib/python3.8/site-packages/tensorflow/python/lib/io/file_io.py", line 77, in _preread_check
    self._read_buf = _pywrap_file_io.BufferedInputStream(
tensorflow.python.framework.errors_impl.NotFoundError: workspace/training_demo/annotations/label_map.pbtxt; No such file or directory
Traceback (most recent call last):
  File "generate_tfrecord.py", line 61, in <module>
    label_map = label_map_util.load_labelmap(ar

# Download and Save Pre-trained Model

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8 /workspace/training_demo/pre-trained-models/

# Copy Model Configuration

In [ ]:
pretrained_model_name = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
custom_model_name = 'my_ssd_resnet50_v1_fpn'
pretrain_model_path = os.path.join('workplace', 'training_demo', 'pre-trained-models', pretrained_model_name, 'pipeline.config')
custom_model_path = os.path.join('workplace', 'training_demo', 'models', custom_model_name)

!cp pretrain_model_path custom_model_path

# Model Training

In [ ]:
# copy script
script_og_path = os.path.join('models', 'research', 'object_detection','model_main_tf2.py')
script_des_path = os.path.join('workplace', 'training_demo')

!cp script_og_path script_des_path

In [ ]:
os.chdir(script_des_path)

command = "python model_main_tf2.py --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(custom_model_path, 
                                                                                                            os.path.join(custom_config_path, 'pipelineconfig'))
!command

# Model Evaluation

In [ ]:
command = "python model_main_tf2.py --model_dir={} --pipeline_config_path={} --checkpoint_dir=models/my_ssd_resnet50_v1_fpn".format(custom_model_path, 
                                                                                                            os.path.join(custom_config_path, 'pipelineconfig'))

!command